In [1]:
%matplotlib widget

import h5py
import scipy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
from scipy import ndimage as ndi

import skimage.morphology as morph
import skimage.measure as measure
import skimage.filters as filt
import skimage.segmentation as seg
import skimage.feature as feature
from skimage.color import label2rgb

from sklearn.metrics import r2_score

## Load data

In [2]:
# Pulling image data from EMD file

f = h5py.File(r'C:\Users\alexl\Documents\Data\20210815 - vaterite HTP\20210808_vaterite_24hr_alpha_30_v3_original.emd', 'r')
htp_images = f['/data/images2048/data'][:]
htp_images.shape

(243, 2048, 2048)

In [3]:
#pulling pixel size of images from metadata

metadata = f['data/images2048/imageparameters']
pixelSize = metadata[10,0]
print(pixelSize)

7.654511e-09


In [4]:
fg, ax = plt.subplots(1,1)
ax.imshow(htp_images[3], cmap='gray')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
# Function to analyze whole image stack
labels = []
properties = {}

def HTP_spherefinder(im_start, im_finish, step_size):
    #start1 = time.time()
    for ii in range(im_start, im_finish, step_size):
        im_data = htp_images[ii]
        elevation_map = filt.sobel(im_data)
        
        # Generate markers (regions from objects) based on threshold value using Otsu's method
        thresh_val = filt.threshold_otsu(im_data)
        markers = np.zeros_like(im_data)
        markers[im_data < thresh_val] = 1
        markers[im_data > thresh_val] = 2
        
        # Watershed transform to fill regions of the elevation map starting from the markers
        segmentation_im = seg.watershed(elevation_map, markers)
        segmentation_im = ndi.binary_fill_holes(segmentation_im - 1)
        
        # Remove small particles and incomplete spheres
        cleaned_im0 = morph.remove_small_objects(segmentation_im, 10000)
        
        # Remove objects touching image border
        cleaned_im1 = seg.clear_border(cleaned_im0, buffer_size=5)
        
        # Watershed segmentation to separate out particles that are touching and overlapping
        distance = ndi.distance_transform_edt(cleaned_im0)
        coords = feature.peak_local_max(distance, footprint=np.ones((200,200)), labels=cleaned_im0)
        mask1 = np.zeros(distance.shape, dtype=bool)
        mask1[tuple(coords.T)] = True
        cleaned_image_markers, _ = ndi.label(mask1)
        labels_ws = seg.watershed(-distance, cleaned_image_markers, mask=cleaned_im1)
        
        # Clean data from the watershed segmentation
        labels_ws0 = morph.remove_small_objects(labels_ws, 1000)

        labels_ws1 = seg.clear_border(labels_ws0, buffer_size=5)
        labels.append(labels_ws1)
        
        # Show labeled spheres and overlay the labels onto original image data
        labeled_spheres, number = ndi.label(labels_ws1)
        # image_label_overlay = label2rgb(labels_ws1, image=im_data, bg_label=0)
        
        # fg, axes = plt.subplots(1, 3, figsize=(8, 3), sharey=True)
        #axes[0].imshow(im_data, cmap=plt.cm.gray)
        #axes[0].contour(cleaned_im0, [0.5], linewidths=0.5, colors='y')
        #axes[1].imshow(cleaned_im1)
        #axes[2].imshow(labels_ws1[ii], cmap=plt.cm.nipy_spectral, interpolation='nearest')
        # axes[3].imshow(image_label_overlay)
        
        # Measure properties from segmented regions
        properties[ii] = measure.regionprops_table(labels_ws1, properties=('area',
                                                                           'major_axis_length',
                                                                           'minor_axis_length',
                                                                           'eccentricity',
                                                                           'label',
                                                                           'centroid'))
        # Index which image in the stack the properties are from
        properties[ii]['image_seq'] = ii
        
        #stop1 = time.time()
        #print('Time to complete: {}'.format(stop1 - start1))  
    return properties, labels


In [6]:
HTP_spherefinder(0,4,1)

({0: {'area': array([], dtype=int32),
   'major_axis_length': array([], dtype=float64),
   'minor_axis_length': array([], dtype=float64),
   'eccentricity': array([], dtype=float64),
   'label': array([], dtype=int32),
   'centroid-0': array([], dtype=float64),
   'centroid-1': array([], dtype=float64),
   'image_seq': 0},
  1: {'area': array([], dtype=int32),
   'major_axis_length': array([], dtype=float64),
   'minor_axis_length': array([], dtype=float64),
   'eccentricity': array([], dtype=float64),
   'label': array([], dtype=int32),
   'centroid-0': array([], dtype=float64),
   'centroid-1': array([], dtype=float64),
   'image_seq': 1},
  2: {'area': array([ 66210, 134020,  63111,  54128,  34432,  27980]),
   'major_axis_length': array([336.74654256, 516.32284095, 303.6183025 , 301.29575963,
          223.33597866, 235.81686418]),
   'minor_axis_length': array([252.8035015 , 367.4078912 , 266.00788983, 229.93947978,
          197.22983807, 168.5891146 ]),
   'eccentricity': array(

## Properties of labeled regions in HTP data

In [7]:
# Record segmented features from multiple image frames into dict
# Note that this example only has one set of measurements that is repeated 
properties = {}
for ii in range(0,4,1):
    properties[ii] = measure.regionprops_table(labels[ii], properties=('area',
                                                               'major_axis_length',
                                                               'minor_axis_length',
                                                               'eccentricity',
                                                               'label',
                                                               'centroid'))
    properties[ii]['image_seq'] = ii

# Create dataframe with the feature properties
pd.set_option('display.max_rows', None)
df = pd.DataFrame([properties[k] for k in range(0,4,1)])
df = df.apply(pd.Series.explode)
df

,area,major_axis_length,minor_axis_length,eccentricity,label,centroid-0,centroid-1,image_seq
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,66210,336.746543,252.803501,0.660617,1,286.689775,419.400362,2
2,134020,516.322841,367.407891,0.702599,4,984.379734,990.106932,2
2,63111,303.618303,266.00789,0.482082,5,1046.623156,729.862148,2
2,54128,301.29576,229.93948,0.646199,7,1374.8733,600.463494,2
2,34432,223.335979,197.229838,0.469169,9,1632.46358,546.029595,2
2,27980,235.816864,168.589115,0.699211,12,1783.105647,1432.853645,2
3,40485,249.448775,207.453568,0.555303,4,598.610028,1669.627047,3
3,49830,272.186812,234.305897,0.508897,5,696.865683,242.958037,3


## Shell thickness analysis from HTP dataset and image labels

In [12]:
#input image number in the stack
jj = 3

cx = properties[jj]['centroid-0']
cx = cx.astype(int)
cy = properties[jj]['centroid-1']
cy = cy.astype(int)

In [17]:
# label image regions
# to make the background transparent, pass the value of `bg_label`,
# and leave `bg_color` as `None` and `kind` as `overlay`
image_label_overlay = label2rgb(labels[jj], image=np.uint16(htp_images[jj]), bg_label=0)

fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(image_label_overlay)
ax.plot(cy, cx, "or")

for xc, yc in zip(cx,cy):
    rect = mpatches.Rectangle((yc-200, xc-200), 400, 400, 
                              fill=False, 
                              edgecolor='red', 
                              linewidth=2)
    ax.add_patch(rect)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
plt.close('all')

#input image number in the stack
jj = 3

cx = properties[jj]['centroid-0']
cx = cx.astype(int) + 300 # account for padded image pixels, (0,0) becomes (0 + padded pixels,0 + padded pixels)
cy = properties[jj]['centroid-1']
cy = cy.astype(int) + 300 # account for padded image pixels

for xc,yc in zip(cx,cy):
    im_padded = np.pad(htp_images[jj], [(300,), (300,)], mode='constant') #padding images with zeros (adjust based on cropped image dimensions)
    labels_padded = np.pad(labels[jj], [(300,), (300,)], mode='constant') #padding labels with zeros
    
    im_cropped = im_padded[xc-300:xc+300, yc-300:yc+300]
    label_cropped = labels_padded[xc-300:xc+300, yc-300:yc+300]
    label_cropped_clean = seg.clear_border(label_cropped, buffer_size=5)
    im_cropped_clean = im_cropped * label_cropped_clean
    
    radial_sum = radial_sum_profile(im_cropped_clean, (300,300))
    radial_fit_funct = radial_sum[0:99] # adjust based on estimated core size
    x_values = np.arange(0,99) # adjust based on estimated core size
    #popt_parab, pcov_parab = scipy.optimize.curve_fit(parabolic_fit, x_values, radial_fit_funct)
    popt_cubic, pcov_cubic = scipy.optimize.curve_fit(cubic_fit, x_values, radial_fit_funct)
    #perr_parab = np.sqrt(np.diag(pcov_parab))
    perr_cubic = np.sqrt(np.diag(pcov_cubic))
    #parab_funct = parabolic_fit(x_values, *popt_parab)
    cubic_funct = cubic_fit(x_values, *popt_cubic)
    #max_x_value = x_values[parab_funct.argmax()]
    max_x_value = x_values[cubic_funct.argmax()]

    fg, axes = plt.subplots(2, 2, figsize=(10, 6))
    axes[0,0].imshow(im_cropped, cmap=plt.cm.gray)
    axes[0,1].imshow(label_cropped)
    axes[1,0].imshow(im_cropped_clean) #mask appkied
    axes[1,1].plot(x_values, radial_fit_funct, '*') #radial sum
    #axes[1,1].plot(x_values, parab_funct, "g") #curve fit function
    axes[1,1].plot(x_values, cubic_funct, "g")
    
    print(cubic_funct.max())
    print(max_x_value)
    print(r2_score(radial_fit_funct, cubic_funct))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

157190.15861141263
55
0.9558547186043568


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

192145.8253862501
69
0.9783385794692504


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

218254.57172445438
0
0.9071458529422404


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

273292.77448895585
7
0.9793468719326901


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

501041.1161443198
72
0.96723789458617


In [9]:
def radial_sum_profile(data, center):
    y, x = np.indices((data.shape))
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    r = r.astype(np.int)

    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel())
    radial_profile = tbin / nr
    return radial_profile 

## Curve fitting functions

In [ ]:
def gaussian(x, amp1, cen1, sigma1):
    return amp1*(1/(sigma1*(np.sqrt(2*np.pi))))*(np.exp((-1.0/2.0)*(((x_array-cen1)/sigma1)**2)))

In [13]:
def parabolic_fit(x, a, b, c):
    return (a*x**2) + (b*x) + c

In [10]:
def cubic_fit(x, a, b, c, d):
    return (a*x**3) + (b*x**2) + (c*x) + d